# Current version : 6.10.a (2024-11-13)

# Libraries and directory (always run)

In [ ]:
### import necessary libraries
import anndata as ad
import anndata
import csv
from datetime import datetime

import geopandas as gpd
from IPython.display import display
import matplotlib as mpl
from matplotlib import animation
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math
import networkx as nx
import numpy as np
import os
import pandas as pd
import pickle
import random
import re
import seaborn as sns
import scanpy as sc
import scanpy.external as sce
from scipy import stats
from scipy.interpolate import CubicSpline
from scipy.io import mmread
from scipy.optimize import curve_fit
import scipy.sparse as sparse
from scipy.stats import pearsonr, pointbiserialr
from shapely.geometry import Polygon, mapping, MultiPolygon
from shapely.ops import unary_union
from skimage import measure
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
import warnings

warnings.filterwarnings("ignore") 
sc.logging.print_header()
sc.set_figure_params(facecolor="white", figsize=(8, 8))
sc.settings.verbosity = 1 # errors (0), warnings (1), info (2), hints (3)
plt.rcParams["font.family"] = "Arial"
sns.set_style("white")

# Note that BANKSY itself is deterministic, here the seeds affect the umap clusters and leiden partition
seed = 1234
np.random.seed(seed)
random.seed(seed)

start_time = datetime.now()

def print_with_elapsed_time(message):
    elapsed_time = datetime.now() - start_time
    elapsed_seconds = elapsed_time.total_seconds()
    print(f"[{elapsed_seconds:.2f} seconds] {message}")

In [ ]:
# print(f"geopandas version: {gpd.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"scanpy version: {sc.__version__}")

In [ ]:
### Directory where the data is stored

# dir = "/mnt/d/Xenium" #Ubuntu
dir = 'D:\\Xenium'
# dir = "/media/volume/data/spatial/hugo/data" #Ubuntu
# dir = "/media/volume/data/spatial/hugo/data/k5" #Ubuntu
# dir = '/media/volume/volume_spatial/hugo/data/test'
# dir = '/media/volume/volume_spatial/hugo/data'

dir_notebook = 'D:\\Jupyter_notebook/Xenium_jupyter_notebook'
# dir_notebook = '/mnt/d/Jupyter_notebook/Xenium_jupyter_notebook'
# dir_notebook = '/media/volume/data/spatial/hugo/notebook'
# dir_notebook = '/media/volume/volume_spatial/hugo/notebook'


In [ ]:
# # # # ### HC ###

# # # # ### make a list of samples and their ids to make the cell names unique
# # # # #samples = ["2505-1__20240404__003359", "2505-2__20240404__003359", "2670-1__20240404__003359", "3159-1__20240321__212855", "3160-1__20240321__212855", "3160-2__20240321__212855"]
# # # # #samples_ids = ["2505-1", "2505-2", "2670-1", "3159-1", "3160-1", "3160-2"]
# # # # #name_dir = 'run1-2'

# # # # # ### Run 1 resegmented
# # # # samples = ["2505-1_subsampled", "2505-2_subsampled", "2670-1_subsampled", 
# # # #    "3159-1_subsampled"
# # # #     , "3160-1_subsampled", "3160-2_subsampled"]
# # # # samples_ids = ["2505-1", "2505-2", "2670-1", 
# # # #               "3159-1", "3160-1", "3160-2"       ]
# # # # name_dir = 'run1-resegment'

# # # # # # #Run 3
# # # # # samples = ["3159-2__20240530__205547", "3159-3__20240530__205547", "3159-4__20240530__205547", "3161-1__20240530__205547", "3161-2__20240530__205547", "3161-3__20240530__205547"]
# # # # # samples_ids = ["3159-2", "3159-3", "3159-4", "3161-1", "3161-2", "3161-3"]
# # # # # name_dir = 'run3-all'

# # # # # Run 3 Habenula
# # # samples = ["3159-2__20240530__205547", "3161-1__20240530__205547"]
# # # samples_ids = ["3159-2", "3161-1"]
# # # name_dir = 'run3-Habenula'

# # # # # # # Run 3 LGN
# # # # samples = ["3159-3__20240530__205547", "3161-2__20240530__205547"]
# # # # samples_ids = ["3159-3","3161-2"]
# # # # name_dir = 'run3-LGN'

# # # # # #Run 3 SC
samples = ["3159-4","3161-3"]
samples_ids = ["3159-4", "3161-3"]
name_dir = 'run3-SC'

# # # # # samples = ["Xenium_Prime_Mouse_Brain_Coronal_FF_outs"]
# # # # # samples_ids = ['Xenium_Prime']
# # # # # name_dir = 'test-prime'

# # # # # # circa1
# # samples = ['circa1_ZT1','circa1_ZT5','circa1_ZT9','circa1_ZT13','circa1_ZT17','circa1_ZT21']
# # samples_ids = ['circa1-ZT1','circa1-ZT5','circa1-ZT9','circa1-ZT13','circa1-ZT17','circa1-ZT21',]
# # name_dir = 'circa1'

# # # ### circa2_
# samples = ['circa2-ZT01','circa2-ZT05','circa2-ZT09','circa2-ZT13','circa2-ZT17','circa2-ZT21']
# samples_ids = ['circa2-ZT01','circa2-ZT05','circa2-ZT09','circa2-ZT13','circa2-ZT17','circa2-ZT21',]
# name_dir = 'circa2'

# # ### circa3_
# samples = ['circa3-ZT01','circa3-ZT05','circa3-ZT09','circa3-ZT13','circa3-ZT17','circa3-ZT21']
# samples_ids = ['circa3-ZT01','circa3-ZT05','circa3-ZT09','circa3-ZT13','circa3-ZT17','circa3-ZT21',]
# name_dir = 'circa3'

# ### circa4-Salk
# samples = ['circa4-Salk-ZT01',
#            'circa4-Salk-ZT05','circa4-Salk-ZT09','circa4-Salk-ZT13','circa4-Salk-ZT17',
#            'circa4-Salk-ZT21',]
# samples_ids = ['circa4-Salk-ZT01',
#                'circa4-Salk-ZT05','circa4-Salk-ZT09','circa4-Salk-ZT13','circa4-Salk-ZT17',
#                'circa4-Salk-ZT21',]
# name_dir = 'circa4-Salk'


# circa4-IGM
# samples_ids = ['circa4-IGM-ZT01','circa4-IGM-ZT05','circa4-IGM-ZT09'
#                    ,'circa4-IGM-ZT13','circa4-IGM-ZT17','circa4-IGM-ZT21',
#                    ]
# samples = ['circa4-IGM-ZT01','circa4-IGM-ZT05','circa4-IGM-ZT09'
#                    ,'circa4-IGM-ZT13','circa4-IGM-ZT17','circa4-IGM-ZT21',
#                    ]
# name_dir = 'circa4-IGM'


### All 5k
# samples_ids = ['circa2-ZT01','circa2-ZT05','circa2-ZT09','circa2-ZT13','circa2-ZT17','circa2-ZT21',
#             'circa4-IGM-ZT01','circa4-IGM-ZT05','circa4-IGM-ZT09',
#             'circa4-IGM-ZT13','circa4-IGM-ZT17','circa4-IGM-ZT21',
#             'circa4-Salk-ZT01','circa4-Salk-ZT05','circa4-Salk-ZT09',
#             'circa4-Salk-ZT13','circa4-Salk-ZT17','circa4-Salk-ZT21',
#                 ]
# samples = ['circa2-ZT01','circa2-ZT05','circa2-ZT09','circa2-ZT13','circa2-ZT17','circa2-ZT21',
#             'circa4-IGM-ZT01','circa4-IGM-ZT05','circa4-IGM-ZT09',
#             'circa4-IGM-ZT13','circa4-IGM-ZT17','circa4-IGM-ZT21',
#             'circa4-Salk-ZT01','circa4-Salk-ZT05','circa4-Salk-ZT09',
#             'circa4-Salk-ZT13','circa4-Salk-ZT17','circa4-Salk-ZT21',
#                ]
# name_dir = 'circa'


# # # # # # ### ALL SAMPLES
# samples = ["2505-1__20240404__003359", "2505-2__20240404__003359", "2670-1__20240404__003359", "3159-1__20240321__212855",
#            "3160-1__20240321__212855", "3160-2__20240321__212855","3159-2__20240530__205547", "3161-1__20240530__205547",
#           "3159-3__20240530__205547", "3161-2__20240530__205547","3159-4__20240530__205547","3161-3__20240530__205547"]

# samples_ids = ["2505-1", "2505-2", "2670-1", "3159-1",
#                "3160-1", "3160-2","3159-2", "3161-1",
#               "3159-3","3161-2","3159-4", "3161-3"
#               ]
# name_dir = 'all-samples'

# samples = ["CONTROLS","GCA"]
# samples_ids = ["CONTROLS","GCA"]
# name_dir = "Human-arteries"

# samples = ["Region1"
#         #    ,"Region2","Region3"
#            ]
# samples_ids = ["Region1"
#             #    ,"Region2","Region3"
#                ]
# name_dir = "MB_test"


# samples = ['S1_Region1']
# samples_ids = ["S1_Region1"]
# name_dir = 'march-test'

# Data importation

In [ ]:
import scanpy as sc

adata = sc.read_h5ad(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_clusters.h5ad.gz")

# Annotations

## Initial annotation

### Automatic initial annotation

In [ ]:
from module.subclustering_Xe import automatic_initial_annotation

adata = automatic_initial_annotation(adata, 'leiden')

### File save

In [ ]:
if 'leiden_colors' in adata_spatial.obs:
    adata_spatial.obs = adata_spatial.obs.drop(columns=['leiden_colors'])

adata_spatial.write(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_clusters.h5ad.gz", compression='gzip')

In [ ]:
adata = sc.read_h5ad(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_clusters.h5ad.gz")

## Cluster check

In [ ]:
adata

In [ ]:
# Generate new numbering base on unique 'cell type'
# all_cell_type = adata.obs['cell type'].unique()
# list_cell_nb = range(0, len(all_cell_type))
# mapping_dict = dict(zip(all_cell_type,list_cell_nb))
# adata.obs['cell_type_newnum'] = adata.obs['cell type'].map(mapping_dict)
# mapping_dict

# all_cell_type = adata.obs['cell_type_auto'].unique()
# list_cell_nb = range(0, len(all_cell_type))
# mapping_dict = dict(zip(all_cell_type,list_cell_nb))
# adata.obs['cell_type_newnum_auto'] = adata.obs['cell_type_auto'].map(mapping_dict)
# mapping_dict

# all_cell_type = adata.obs['cell_type_final'].unique()
# list_cell_nb = range(0, len(all_cell_type))
# mapping_dict = dict(zip(all_cell_type,list_cell_nb))
# adata.obs['cell_type_newnum_final'] = adata.obs['cell_type_final'].map(mapping_dict)
# mapping_dict

# all_cell_type = adata.obs['mmc:subclass_name'].unique()
# list_cell_nb = range(0, len(all_cell_type))
# mapping_dict = dict(zip(all_cell_type,list_cell_nb))
# adata.obs['mmc:subclass_num'] = adata.obs['mmc:subclass_name'].map(mapping_dict)
# mapping_dict

# all_cell_type = adata.obs['mmc:class_name'].unique()
# list_cell_nb = range(0, len(all_cell_type))
# mapping_dict = dict(zip(all_cell_type,list_cell_nb))
# adata.obs['mmc:class_num'] = adata.obs['mmc:class_name'].map(mapping_dict)
# mapping_dict


In [ ]:
# adata.obs.groupby('cell type')['cell type'].count()
adata.obs['cell_type_final'].value_counts().sort_index()

In [ ]:
### Check clusters one by one to see if they are present in all sample and which would need subclustering

cluster_to_use = 'cell_type_newnum_auto'
# cluster_to_use = 'cell_type_newnum_auto_sub'
# cluster_to_use = 'cell_type_newnum_final'
# cluster_to_use = 'mmc:subclass_num'
# cluster_to_use = 'mmc:class_num'

### Generate a color palette for the clusters - to make color stay consistent across samples
adata.obs[cluster_to_use] = adata.obs[cluster_to_use].astype(str)

# Create a palette with a unique color for each cluster
num_clusters = len(adata.obs[cluster_to_use].astype(int).unique())
palette = sns.color_palette("tab20b", n_colors=num_clusters +1)

# Map each 'leiden' value to a color
adata.obs['leiden_colors'] = adata.obs[cluster_to_use].astype(int).apply(lambda x: palette[x])

# Map all cells
fig, axs = plt.subplots(6,3,figsize=(15, 20))
axs = axs.flatten()
clusters_plot = {"14":'black', ### For VLMC
    '0': 'lightcoral', "1" : 'forestgreen', '2':'red', "3":'purple', "4":"yellow",
    # '5': 'lightcoral', "6" : 'forestgreen', "7":'red', "8":'purple', "9":"yellow"
    # '10': 'lightcoral',"11": 'forestgreen', '12':'red', "13":'purple', "14":"yellow", "15": "blue",
    # '16': 'lightcoral',"17": 'forestgreen', '18':'red', "19":'purple', "20":"yellow"
    # '21': 'lightcoral',"22": 'forestgreen', '23':'red', "":'purple', "":"yellow",
    # '24': 'lightcoral',"25": 'forestgreen', '26':'red', "27":'purple', "28":"yellow",
    # '29': 'lightcoral',"30": 'forestgreen', '31':'red', "32":'purple', "33":"yellow",
    # '34': 'lightcoral',"35": 'forestgreen', '36':'red', "37":'purple', "38":"yellow", "39": "blue",
    # '40': 'lightcoral',"41": 'forestgreen', '42':'red', "43":'purple', "44":"yellow", "45": "blue",
    # '46': 'lightcoral',"47": 'forestgreen', '48':'red', "49":'purple', "50":"orange", "51": "blue",
    # '52': 'lightcoral',"53": 'forestgreen', '54':'red', "55":'purple', "56":"yellow", "57": "blue",
    # '58': 'lightcoral',"59": 'forestgreen', '60':'red', "61":'purple', "62":"yellow", "63": "blue",
    # '64': 'lightcoral',"65": 'forestgreen', '66':'red', "67":'purple', "68":"yellow", "69": "blue",
    # '70': 'lightcoral',"71": 'forestgreen', '72':'red', "73":'purple', "74":"yellow", "75": "blue",
    # '76': 'lightcoral',"77": 'forestgreen', '78':'red', "79":'purple', "80":"yellow", "81": "blue",
    # '82': 'lightcoral',"83": 'forestgreen', '84':'red', "85":'purple', "86":"orange", "87": "blue",
    # '88': 'lightcoral',"89": 'forestgreen', '90':'red', "91":'purple', "92":"yellow", "93": "blue",
    # '94': 'lightcoral',"95": 'forestgreen', '96':'red', "97":'purple', "98":"yellow", "99": "blue",
    # '100':'lightcoral',"101": 'forestgreen', '102':'red', "103":'purple', "104":"yellow", "105": "blue",


    '13': 'lightcoral',"15": 'forestgreen', '16':'red', "":'purple', "32":"orange",'':'blue',
    # '59': 'lightcoral',"60": 'forestgreen', '64':'red', "65":'purple', "61":"yellow",
}

for idx, sample in enumerate(samples_ids):
    adata_sel = adata[(adata.obs['sample'] == sample)]
    for cluster_id in adata_sel.obs[cluster_to_use].unique():
        cluster_data = adata_sel.obs[adata_sel.obs[cluster_to_use] == cluster_id]
        colors = clusters_plot[cluster_id] if cluster_id in clusters_plot else "none" ### for selected clusters in cluster_plot
        # colors = cluster_data['leiden_colors'].unique()[0] ### uncomment for all clusters
        axs[idx].scatter(cluster_data['x_centroid'], cluster_data['y_centroid'], color=colors, s=0.01, label=cluster_id)
        axs[idx].get_xaxis().set_visible(False)
        axs[idx].get_yaxis().set_visible(False)
        axs[idx].set_title(f"Sample {sample}")
del adata_sel

In [ ]:
from matplotlib.pyplot import rc_context
with rc_context({"figure.figsize": (10, 10)}):
    sc.pl.umap(
        adata,
        color="cell_type_auto_sub",
        add_outline=True,
        legend_loc="on data",
        legend_fontsize=12,
        legend_fontoutline=2,
        frameon=False,
        title="clustering of cells",
        palette="tab20",
    )

In [ ]:
# To rename a cell type in case of typo or mistakes (easier than re-running)
rename_subclass = {
'AbC':'ABC',
'L2/3 IT RSP Glut' : 'L2 3 IT RSP Glut' ,
'L2 3 IT PIR-ENTl Glut':'L2 3 IT PIR ENTl Glut',
}
                  
adata.obs['cell_type_final'] = adata.obs['cell_type_final'].apply(lambda x: rename_subclass[x] if x in rename_subclass else x)
adata.obs['cell_type_final'].unique()

In [ ]:
# To rename a cell type in case of typo or mistakes (easier than re-running)
rename_subclass = {'L4/5 IT CTX': 'L4 5 IT CTX',
                   'L2/3 IT CTX': 'L2 3 IT CTX',
                   'CA1-ProS':  'CA1 ProS',
                   'STN-PSTN': 'STN PSTN',
                   'SUB-ProS': 'SUB ProS',
                   'L2/3 IT RSP': 'L2 3 IT RSP',
                   'L2/3 UT RSP': 'L2 3 IT RSP',
                   'L2/3 IT ENT': 'L2 3 IT ENT',
                   'L5/6 IT TPE-ENT': 'L5 6 IT TPE ENT',
                   'CLA-EPd-CTX': 'CLA EPd CTX',
                   'STR D1/D2': 'STR D1D2 GABA',
                   'STR D1/D2 GABA': 'STR D1D2 GABA',
                   'L2/3 IT PIR-ENTI': 'L2 3 IT PIR ENTI',
                   'L2/3 PIR-ENTI': 'L2 3 IT PIR ENTI',
                   'L2 3 IT PIR-ENTI': 'L2 3 IT PIR ENTI',
                   'Pvalb-Sst GABA': 'interneurons GABA',
                   'L4/5 IT CTX': 'L4 5 IT CTX',
                   'MEA-BST GABA': 'MEA BST GABA',
                   'TRS-BAC Glut': 'TRS BAC Glut',
                   'STR-PAL GABA': 'STR PAL GABA',
                   'PAL-STR GABA-Chol': 'PAL STR GABA Chol',
                   'RT-ZI Glut': 'RT ZI Glut',
                   'CLA-EPd-CTX Glut':  'CLA EPd CTX Glut',
                   'RT-ZI': 'RT ZI',
                   'CA1-ProS': 'CA1 ProS',
                   'Chor':'CHOR',
                   'L2/3 IT':  'L2 3 IT',
                   'L2/3 IT CTX': 'L2 3 IT CTX',
                   'Pvalb-Sst': 'interneurons GABA',
                   'IT EP-CLA': 'IT EP CLA',
                   'L4 RSP-ACA': 'L4 RSP ACA',
                   'Sub-ProS': 'SUB ProS',
                   'Interneurons GABA':'interneurons GABA',

                   
                   
                  }
adata.obs['cell type'] = adata.obs['cell type'].apply(lambda x: rename_subclass[x] if x in rename_subclass else x)
adata.obs['cell type'].unique()

In [ ]:
adata.obsm['X_pca'] = adata.obsm['reduced_pc_20']
adata.obsm['X_umap'] = adata.obsm['reduced_pc_20_umap']

In [ ]:
if 'leiden_colors' in adata.obs:
    adata.obs = adata.obs.drop(columns=['leiden_colors'])

# adata.write(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_MMC_Banksy_clusters.h5ad.gz", compression='gzip')
adata.write(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_MMC_Banksy_clusters_combined.h5ad.gz", compression='gzip')

In [ ]:
adata = sc.read_h5ad(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_MMC_Banksy_clusters_combined.h5ad.gz")

## Subclustering

In [ ]:
# adata.obs.groupby('cell type')['cell type'].count()
adata.obs['cell_type_newnum_auto'].value_counts()

### Automatic subclustering

In [ ]:
from module.subclustering_Xe import auto_subclustering2

auto_subclustering2(adata_to_sub = adata, ###
                    all_types =  'all', ### 'all' or ['0','1'] for selected number of clusters
                    Clusters_to_use = 'cell_type_newnum_auto',
                    resolution = 0.05)

In [ ]:
from subclustering_Xe import cluster_table

cont_tab, cont_tab_sub, cluster_df = cluster_table(adata_to_use = adata,
                                                   Clusters_to_use = 'cell_type_newnum_auto_sub',
                                                   sort_order='Cell Count',
                                                   sort_ascend = True
                                                   )

cluster_df

In [ ]:
tst_dict = cont_tab_sub.T.idxmax(axis=0).to_dict()
tst_dict = dict(zip(cluster_df.index, cluster_df['Celltype']))
tst_dict

#### Final clusters

In [ ]:
# Final clusters
rename_subclass = {"31": 'CLA EPd CTX Glut',
 "25": 'L6 IT CTX Glut',
 "34": 'L4 5 IT CTX Glut',
 "35": 'L2 3 IT CTX Glut',
 "9": 'L2 3 IT PIR-ENTl Glut',
 "39": 'CA1 ProS Glut',
 "27": 'CA3 Glut',
 "44": 'L2/3 IT RSP Glut',
 "45": 'L4 RSP ACA Glut',
 "36": 'L5 ET CTX Glut',
 "28": 'CA2 FC IG Glut',
 "53": 'NLOT Glut',
 "32": 'L6b CTX Glut',
 "33": 'L6 CT CTX Glut',
 "37": 'L5 NP CTX Glut',
 "47": 'HPF CR Glut',
 "43": 'DG Glut',
 "51": 'OB STR CTX Inh IMN',
 "26": 'Vip Gaba',
 "29": 'Lamp5 Gaba',
 "12": 'Pvalb Gaba',
 "11": 'Sst Gaba',
 "17": 'STR D1D2 Gaba',
 "24": 'PAL STR Gaba Chol',
 "14": 'STR D1D2 Gaba',
 "18": 'STR D1D2 Gaba',
 "48": 'STR D1D2 Gaba',
 "38": 'STR PAL Gaba',
 "52": 'LSX Gaba',
 "16": 'CEA BST Gaba',
 "15": 'CEA BST Gaba',
 "23": 'RT ZI Gaba',
 "55": 'SCH Gaba',
 "54": 'BST po Glut',
 "49": 'TRS BAC Glut',
 "10": 'COAa PAA MEA Glut',
 "1": 'VMH Glut',
 "21": 'AHN RCH LHA Glut',
 "5": 'PVH SO PVa Glut',
 "42": 'MH Glut',
 "41": 'LH Glut',
 "56": 'AD Glut',
 "4": 'TH Glut',
 "50": 'MB Gaba',
 "8": 'Astro TE',
 "0": 'Tanycyte',
 "2": 'Ependymal',
 "40": 'Choroid',
 "20": 'OPC',
 "3": 'Oligodendrocyte',
 "13": 'ABC',
 "19": 'VLMC',
 "30": 'Pericyte',
 "46": 'Endothelial',
 "6": 'Endothelial',
 "7": 'Microglia',
 "22": 'Microglia'}

adata.obs['cell_type_final'] = adata.obs['cell_type_newnum_auto_sub'].apply(lambda x: rename_subclass[x] if x in rename_subclass else x)
adata.obs['cell_type_final'].unique()

all_cell_type = adata.obs['cell_type_final'].unique()
list_cell_nb = range(0, len(all_cell_type))
mapping_dict = dict(zip(all_cell_type,list_cell_nb))
adata.obs['cell_type_newnum_final'] = adata.obs['cell_type_final'].map(mapping_dict)
mapping_dict

In [ ]:
### To rename only some cell types
rename_subclass = dict(zip(adata.obs['cell_type_newnum_final'],adata.obs['cell_type_final']))
rename_subclass_temp = {
13:'STR PAL Gaba',
14:'L2 3 IT PIR ENTl Glut',
20 : 'L2 3 IT CTX',
22:'STR D1D2 Gaba',
31: 'OB STR CTX Inh IMN',
32 : 'L4 5 IT CTX Glut',
33 :'CLA EPd CTX Glut',
35 : 'PAL STR Gaba Chol',
36 :'BST po Glut',
43  : 'RT ZI Gaba',
44 :'TRS BAC Glut',
45 :'L2 3 IT RSP Glut',
46: 'CA2 FC IG Glut',
54:'CA1 ProS Glut',
}
rename_subclass.update(rename_subclass_temp)
adata.obs['cell_type_final'] = adata.obs['cell_type_newnum_final'].apply(lambda x: rename_subclass[x] if x in rename_subclass else x)
adata.obs['cell_type_final'].unique()

all_cell_type = adata.obs['cell_type_final'].unique()
list_cell_nb = range(0, len(all_cell_type))
mapping_dict = dict(zip(all_cell_type,list_cell_nb))
adata.obs['cell_type_newnum_final'] = adata.obs['cell_type_final'].map(mapping_dict)
mapping_dict

In [ ]:
### Correlation map
# Clusters_to_use = 'cell_type_newnum_auto_sub'
# cont_tab = pd.crosstab(adata_filter.obs[Clusters_to_use], adata_filter.obs['mmc:class_name'], normalize="index")
# cont_tab = cont_tab.loc[:, cont_tab.sum(axis=0) > 0.1] 
plt.figure(figsize=(40, 20))
sns.heatmap(cont_tab.T, annot=True, cmap="YlGnBu", fmt=".1f", cbar = False)

In [ ]:
### Correlation map
# cont_tab_sub = pd.crosstab(adata_filter.obs[Clusters_to_use],adata_filter.obs['mmc:subclass_name'], normalize="index")
# cont_tab_sub = cont_tab_sub.loc[:, cont_tab_sub.sum(axis=0) > 0.05] 
plt.figure(figsize=(40, 50))
sns.heatmap(cont_tab_sub.T, annot=True, cmap="YlGnBu", fmt=".1f", cbar = False)

In [ ]:
cont_tab_sub.T[2].sort_values(ascending=False).head(15)

### Manual subclustering

In [ ]:
# Clusters_to_use = 'cell_type_newnum_auto'
# Clusters_to_use = 'cell_type_newnum_auto_sub'
Clusters_to_use = 'cell_type_newnum_final'
# Clusters_to_use = 'cell_type_newnum'
# Clusters_to_use = 'leiden'
adata_filter = adata

In [ ]:
### Correlation map
cont_tab = pd.crosstab(adata_filter.obs[Clusters_to_use], adata_filter.obs['mmc:subclass_name'], normalize="index")
cont_tab = cont_tab.loc[:, cont_tab.sum(axis=0) > 0.1] 
plt.figure(figsize=(40, 40))
sns.heatmap(cont_tab.T, annot=True, cmap="YlGnBu", fmt=".1f", cbar = False)

In [ ]:
### Select a cluster to subcluster
cluster_to_sub = "1"

adata_subcluster = adata_filter[adata_filter.obs[Clusters_to_use] == cluster_to_sub]
adata_subcluster.obs[Clusters_to_use].sample() , adata_subcluster.shape

In [ ]:
sc.pp.pca(adata_subcluster)
sc.pp.neighbors(adata_subcluster)
sc.tl.umap(adata_subcluster)

In [ ]:
# extract pca coordinates
X_pca = adata_subcluster.obsm['X_pca'] 

### Kmeans clustering
### You can choose the number of clusters by uncommenting n_clusters option
# kmeans = KMeans(#n_clusters=4,
#                 random_state=0).fit(X_pca) 
# adata_subcluster.obs['kmeans'] = kmeans.labels_.astype(str)

sc.tl.leiden(adata_subcluster, resolution = 0.2)

In [ ]:
clustering_method = 'leiden'

In [ ]:
from matplotlib.pyplot import rc_context
with rc_context({"figure.figsize": (5, 5)}):
    sc.pl.umap(
        adata_subcluster,
        color=clustering_method,
        add_outline=False,
        legend_loc="on data",
        legend_fontsize=12,
        legend_fontoutline=2,
        frameon=False,
        palette="tab20",
    )
# sc.pl.pca(adata_subcluster,
#          color=clustering_method,
#          palette="tab20",
#          )

In [ ]:
### Number of cells per clusters
max_clust = len(adata_subcluster.obs[clustering_method].unique())
for i in range(0, max_clust):
    count = adata_subcluster.obs[clustering_method].value_counts().iloc[i]
    print(f"Cluster {i} : {count} cells")

# adata_subcluster.obs['leiden'].sample(10)

In [ ]:
### Correlation map of subclusters
cont_tab = pd.crosstab(adata_subcluster.obs[clustering_method], adata_subcluster.obs['mmc:class_name'], normalize="index")
plt.figure(figsize=(10, 8))
sns.heatmap(cont_tab.T, annot=True, cmap="YlGnBu", fmt=".1f", cbar=False)

In [ ]:
### Correlation map of subclusters
cont_tab = pd.crosstab(adata_subcluster.obs[clustering_method], adata_subcluster.obs['mmc:subclass_name'], normalize="index")
cont_tab = cont_tab.loc[:, cont_tab.sum(axis=0) > 0.05]
plt.figure(figsize=(15, 15))
sns.heatmap(cont_tab.T, annot=True, cmap="YlGnBu", fmt=".1f", cbar=False) 

In [ ]:
# adata_subcluster.obs[clustering_method] = adata_subcluster.obs[clustering_method].astype(str)
# # sc.tl.dendrogram(adata_subcluster, groupby = cluster_to_use, n_pcs=None, use_rep=None, var_names=None, use_raw=None, cor_method='pearson', linkage_method='complete', optimal_ordering=False, key_added=None)
# sc.tl.rank_genes_groups(adata_subcluster, groupby=clustering_method, method="wilcoxon", tie_correct = True, dendrogram = False)
# sc.pl.rank_genes_groups_dotplot(adata_subcluster, groupby=clustering_method, standard_scale="var", n_genes=2, dendrogram = False)

In [ ]:
### Generate a color palette for the clusters - to make color stay consistent across samples
adata_subcluster.obs[clustering_method] = adata_subcluster.obs[clustering_method].astype(int)

# Create a palette with a unique color for each cluster
num_clusters = len(adata_subcluster.obs[clustering_method].unique())
palette = sns.color_palette("tab20", n_colors=num_clusters)

# Map each 'leiden' value to a color
adata_subcluster.obs['kmeans_colors'] = adata_subcluster.obs[clustering_method].apply(lambda x: palette[x])

# Mapping of clusters
fig, axs = plt.subplots(6,3,figsize=(15, 20))
axs = axs.flatten()
clusters_plot = {
    0: 'orchid', 1: 'forestgreen', 2: 'black', 3:'red', 
    4:'cyan', 5:'blue', 6:'darkorange', 7:'coral',
    8:'forestgreen', 9: 'coral',10:'red', 11:'cyan',
    12:'blue',13:'darkorange',14:'black'
}

for idx, sample in enumerate(samples_ids):
    adata_sel = adata_subcluster[(adata_subcluster.obs['sample'] == sample)]
    for cluster_id in adata_sel.obs[clustering_method].unique():
        cluster_data = adata_sel.obs[adata_sel.obs[clustering_method] == cluster_id]
        colors = clusters_plot[cluster_id] if cluster_id in clusters_plot else "none"
        # colors= cluster_data['kmeans_colors'].unique()[0]
        axs[idx].scatter(cluster_data['x_centroid'], cluster_data['y_centroid'], color=colors, s=2, label=cluster_id)
        axs[idx].set_title(f"Sample {sample}")

In [ ]:
adata_subcluster.obs['new_cluster'] = clustering_method
adata_subcluster.obs['new_cluster2'] = adata_subcluster.obs[Clusters_to_use].astype("str") + '.' + adata_subcluster.obs[clustering_method].astype("str")
adata_subcluster.obs[['cell_id','new_cluster2']].sample(2)

In [ ]:
# Use this dictionnary to rename ['cell type'] with the new appropriate cell type for the subcluster. Follow the format. One subcluster at the time.
rename_subclass = {
 f'{cluster_to_sub}.0':'VLMC',
f'{cluster_to_sub}.1':'ABC',
f'{cluster_to_sub}.2':'AbC',
f'{cluster_to_sub}.3':'ABC',
f'{cluster_to_sub}.4':'',
f'{cluster_to_sub}.5':'',
f'{cluster_to_sub}.6':'',
f'{cluster_to_sub}.7':'',
f'{cluster_to_sub}.8' :'',
f'{cluster_to_sub}.9' :'',
f'{cluster_to_sub}.10':'',
f'{cluster_to_sub}.11':'',
f'{cluster_to_sub}.12':'',
f'{cluster_to_sub}.13':'',
f'{cluster_to_sub}.14':'',
f'{cluster_to_sub}.15':'',
f'{cluster_to_sub}.16':'',
f'{cluster_to_sub}.17':'',
}

adata_subcluster.obs['cell_type_final'] = adata_subcluster.obs['new_cluster2'].map(rename_subclass)

# Create a dictionary to map old values to new values
mapping_dict = dict(zip(adata_subcluster.obs['cell_id'], adata_subcluster.obs['cell_type_final']))

# Use .map() function to rename cell contents in 'col1' based on mapping dictionary
adata.obs['cell_type_final'] = adata.obs.apply(lambda x: mapping_dict[x['cell_id']] if x['cell_id'] in mapping_dict else x['cell_type_final'],axis = 1)

In [ ]:
adata.obs["cell_type_final"].sample(5)


In [ ]:
adata = adata[adata.obs['cell type'] != 'Undefined']

<font size="6"><span style="color:red">From here, go back to process the other cluster if needed </span></font>

## File Save (and load)

In [ ]:
all_cell_type = adata.obs['cell_type_final'].unique()
list_cell_nb = range(0, len(all_cell_type))
mapping_dict = dict(zip(all_cell_type,list_cell_nb))
adata.obs['cell_type_newnum_final'] = adata.obs['cell_type_final'].map(mapping_dict)
mapping_dict

In [ ]:
# adata.obs.groupby('cell_type_final')['cell_type_final'].count().sort_values()
adata.obs['cell_type_final'].value_counts()

In [ ]:
if 'leiden_colors' in adata.obs:
    adata.obs = adata.obs.drop(columns=['leiden_colors'])
    # adata.obs = adata.obs.drop(columns=['cell_class_auto_sub'])


adata.write(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_MMC_Banksy_annotated_combined.h5ad.gz", compression='gzip')
adata.obs.to_csv(f"{dir_notebook}/csv/{name_dir}/{name_dir}_MMC_Banksy_annotated_combined.csv.gz", compression='gzip')

In [ ]:
adata = sc.read_h5ad(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_MMC_Banksy_annotated_combined.h5ad.gz")

# Automap

## Data pre-processing

In [ ]:
# testdf = pd.read_csv('Xenium-data-coordinates-CTX.csv')
testdf = pd.read_csv(f'{dir_notebook}/csv/{name_dir}/{name_dir}_MMC_Banksy_annotated_combined.csv.gz')
testdf.shape

testdf = testdf.filter(['cell_id','sample','x_centroid','y_centroid','cell_type_final','cell_type_newnum_final'], axis=1)
### Only keep necessary columns

In [ ]:
testdf.sample()

In [ ]:
testdf['cell_type_final'].unique()

In [ ]:
# Simplify names for mapping
rename_subclass = {
'Oligo':'Undefined',
'Oligodendrocyte':'Undefined',
'Astro':'Undefined',
'Astro TE':'Undefined',
'Astro NT':'Undefined',
'ABC':'Undefined',
'Endo':'Undefined',
'Endothelial':'Undefined',
'OPC':'Undefined',
'Pericyte':'Undefined',
'Tanycyte':'Ependymal',
'Microglia':'Undefined', 
'Undefined':'Undefined',
'CHOR':'Undefined',
'HPF CR':'Undefined',

'HY GABA':'HY',
'HY Glut':'HY',
'RT ZI GABA':'HY',
'RT ZI':'HY',
'SO Glut':'HY',
'AHN Glut':'HY',
'AHN':'HY',
'LHA Glut':'HY',
'PVH Glut':'HY',
'HY GABA': 'HY',
'HY Gaba': 'HY',
'HY Glut':'HY',
'Mix HY' : 'HY',
'VMH Glut':'VMH',

'AMY GABA':'AMY',
'AMY Glut':'AMY',

'TH Glut':'TH',
'PVT Glut': 'TH',
'PVT':'TH',
'AD':'TH',

'MB GABA':'MB',
'MB DOPA':'MB',
'MB Dopa':'MB',
'MB Glut':'MB',

'L4 5 IT CTX':'CTX',
'L5 IT CTX':'CTX',
'L2 3 IT PIR ENTI':'CTX',
'L5 6 IT TPE ENT':'CTX',
'L2 3 IT CTX':'CTX', 
'L5 ET CTX':'CTX',
'L2 3 IT':'CTX',
'L6 IT CTX':'CTX',
'L6 CT CTX':'CTX',
'L5 NP CTX':'CTX',
'L4 RSP ACA':'CTX',
'L4 5 IT Glut':'CTX',
'L2 3 4 5 IT CTX':'CTX',
'L2 3 IT RSP':'CTX',
'L5 CTX' : 'CTX',
'L6 CTX' : 'CTX',
'L2 3 CTX': 'CTX',
'L4 5 CTX': 'CTX',
'L6b CTX':'CTX',
'L6b CTX':'CTX',
'L2 3 IT PIT ENTI':'CTX',
'Mix Cortex':'CTX',
'NP':'CTX',
'CLA EPd CTX Glut':'CTX',
'IT EP CLA':'CTX',
'CLA':'CTX',
'CLA EP':'CTX',
'interneurons GABA':'Undefined',
'interneurons':'Undefined',
'Sncg Gaba': 'Undefined',
'Pvalb Gaba':'Undefined',
'Vip Gaba':'Undefined',
'Sst Gaba':'Undefined',
'Lamp5 Gaba':'Undefined',

'SCG Gaba':'SCH',
'NLOT Glut':'NLOT',

'STR D1D2 GABA':'STR',
'STR GABA':'STR',
'STR D1D2':'STR',
'STRv' : 'STR',
'GPe GABA' : 'GPe',
'STRv GABA':'STRv',
'BST GABA':'BST',
'GP GABA':'PAL',
'STR PAL Gaba' : 'STR',
    
'TRS BAC Glut':'TRS',

'LSX GABA':'LSX',

'NP SUB':'HIPP',
'CA1 ProS':'HIPP',
'CA3':'HIPP',
'DG':'HIPP',
'DG Glut':'HIPP',
'SUB ProS':'HIPP',
'NP SUB Glut':'HIPP',
'CA1 ProS Glut':'HIPP',
'CA3 Glut':'HIPP',
'DG Glut':'HIPP',
'CA2 FC IG Glut':'HIPP',
'SUB ProS Glut':'HIPP',

'ARH GABA':'ARH',
'PF Glut': 'MB',
'LA Glut': 'AMY',
'MEA Glut':'AMY', 
'ZI Gaba':'HY', 
'STRv PAL Gaba':'STRv', 
'L23 PIR ENTl Glut':'CTX',
'L4 CTX Glut':'CTX',
'L23 CTX Glut':'CTX',
'L5 CTX Glut':'CTX',
'STR D1D2 Gaba':'STR',
'L6b CTX Glut':'CTX',
'L6 CTX Glut':'CTX',
'MEA Gaba':'AMY',
'MH Glut':'MH', 
'LH Glut':'LH',
'L23 RSP Glut':'CTX',
'STR Gaba':'STR',
'BST Gaba':'BST', 
'SC Glut':'SC',
'MM Glut':'MM', 
'SCH Gaba':'SCH',
'LSX Gaba':'LSX',
'SC Gaba':'MB',
'LGv Gaba':'MB',
'APN Glut':'TH',
'AV Glut':'HY',
'PRT Gaba':'MB',
'BST Glut':'BST',
'AD Glut':'HY',
'SN Gaba':'MB',
'Pineal Glut':'MB',
'SN Dopa':'MB',
'PAG Glut':'MB',
'STRv':'STR',
'BST':'HY',

'ARH Glut': 'ARH',
'L2 3 IT PIR ENTl Glut':'CTX',
'AMY Gaba':'AMY',
'CLA EPd Glut':'CTX',
'PAL STR Gaba':'STRv',
'L6 IT CTX Glut':'CTX',
'L2345 IT CTX Glut':'CTX',
'L6 CT CTX Glut':'CTX',
'L2 3 IT RSP Glut':'CTX',
'L4 RSP ACA Glut':'CTX',
'OB CR Glut':'HIPP',

'PVH SO PVa Glut': "HY",
'AHN RCH LHA Glut' : "HY",
'RT ZI Gaba' : "HY",
'PAL STR Gaba Chol': "PAL",
'L4 5 IT CTX Glut' : "CTX",
'L2 3 IT CTX Glut' : "CTX",
'L5 ET CTX Glut' : "CTX",
'L5 NP CTX Glut' : "CTX",
'IT EP CLA Glut' : "CTX",
'Choroid' : "Undefined",
'LH' : 'TH',
'MH' : "TH",
'HPF CR Glut' : 'HIPP',
'MB Gaba' : "MB",
'OB STR CTX Inh IMN' : "HIPP",
'BST po Glut' : "BST"

}
testdf['cell_type_final'] = testdf['cell_type_final'].apply(lambda x: rename_subclass[x] if x in rename_subclass else x)
testdf = testdf[testdf['cell_type_final'] != 'Undefined']
testdf = testdf[testdf['cell_type_final'] != 'undefined']
testdf['cell_type_final'].unique()

In [ ]:
sample_ids = testdf['sample'].unique()
sample_ids

In [ ]:
from automap import knn_mst_clustering
from automap import assign_chunk_labels
from automap import fill_any_chunks,fill_empty_chunks
from automap import grid_to_geojson_with_scaling
import json


count = 0
for sample_to_map in sample_ids:

    count +=1
    print(f'{datetime.now()}: automap of {sample_to_map} ({count}/{len(samples_ids)})')

    df = testdf[testdf['sample']==sample_to_map]
    all_cell_type = df['cell_type_final'].unique()
    list_cell_nb = range(0, len(all_cell_type))
    mapping_dict = dict(zip(all_cell_type,list_cell_nb))
    df['cell_type_newnum_final'] = df['cell_type_final'].map(mapping_dict)

    # Multiply centroid coordinates by 10 to convert to pixel coordinates
    df['x_pixel'] = df['x_centroid'] * 10
    df['y_pixel'] = df['y_centroid'] * 10

    ### Generate a color palette for the clusters - to make color stay consistent across samples
    df['cell_type_newnum_final'] = df['cell_type_newnum_final'].astype(str)

    ### Establish neighbors classifiers

    K = 10
    X = df[['x_pixel', 'y_pixel']].values
    y = df['cell_type_newnum_final'].astype(int).values
    knn = KNeighborsClassifier(n_neighbors=K)
    knn.fit(X, y)
    df['knn_celltype'] = knn.predict(X)

    data = df
    valid_celltypes = data['knn_celltype'].value_counts()[data['knn_celltype'].value_counts() > 100].index
    data = data[data['knn_celltype'].isin(valid_celltypes)]
    unique_cell_types = sorted(data['knn_celltype'].unique())

    for cell_type in unique_cell_types:
        print(f"Processing cell type {cell_type}...")
        knn_mst_clustering(data, cell_type=cell_type, k=10)

    # Save the updated dataframe with cluster labels
    # data.to_csv('Xenium-data-coordinates-CTX-labeled.csv', index=False)
    print("Data with cluster labels saved.")

    data['knn_celltype'].value_counts()
    data['cluster_label'].isna().sum()

    # Get the value counts of each category
    value_counts = data['cluster_label'].value_counts()

    # Create a mapping based on the rank of value counts
    mapping_drg = {label: idx for idx, label in enumerate(value_counts.index, 1)}

    # Map the categorical strings to numbers based on the value counts
    data['cluster_label_numeric'] = data['cluster_label'].map(mapping_drg)
    data['cluster_label_numeric'].unique()
    data = data.dropna(subset=['cluster_label_numeric'])
    data['cluster_label_numeric'].isna().sum()

    ### Pixelize section and assign label
    chunk_size = 400  # Example chunk size
    threshold = 0.5  # Example threshold
    grid, x_bins, y_bins = assign_chunk_labels(data, chunk_size, threshold)


    filled_grid, x_bins, y_bins = fill_empty_chunks(grid, x_bins, y_bins, 4)
    filled_grid, x_bins, y_bins = fill_empty_chunks(filled_grid, x_bins, y_bins, 3)
    filled_grid, x_bins, y_bins = fill_empty_chunks(filled_grid, x_bins, y_bins, 4)
    filled_grid, x_bins, y_bins = fill_empty_chunks(filled_grid, x_bins, y_bins, 3)
    filled_grid, x_bins, y_bins = fill_any_chunks(filled_grid, x_bins, y_bins,4)
    filled_grid, x_bins, y_bins = fill_empty_chunks(filled_grid, x_bins, y_bins, 3)
    filled_grid, x_bins, y_bins = fill_empty_chunks(filled_grid, x_bins, y_bins, 2)
    filled_grid, x_bins, y_bins = fill_empty_chunks(filled_grid, x_bins, y_bins, 2)
    filled_grid, x_bins, y_bins = fill_empty_chunks(filled_grid, x_bins, y_bins, 3)
    filled_grid, x_bins, y_bins = fill_any_chunks(filled_grid, x_bins, y_bins,4)
    filled_grid, x_bins, y_bins = fill_any_chunks(filled_grid, x_bins, y_bins,3)
    filled_grid, x_bins, y_bins = fill_any_chunks(filled_grid, x_bins, y_bins,3)

    # Create the mapping as before
    mapping = data.groupby('knn_celltype')['cell_type_final'].agg(lambda x: x.value_counts().idxmax()).to_dict()

    # Create a new column 'knn_celltype_label' using the mapping
    data['knn_celltype_label'] = data['knn_celltype'].map(mapping)
    data['cell_type_newnum_final'] = data['cell_type_newnum_final'].astype(int)

    # Display the dataframe to verify
    display(data[['knn_celltype', 'knn_celltype_label']].sample(3))

    ### Export geojson
    value_counts = data[['knn_celltype', 'cell_type_final']].value_counts()
    value_counts_df = value_counts.reset_index(name='count')
    most_common_pairs = value_counts_df.groupby('knn_celltype').apply(lambda x: x.nlargest(1, 'count'))
    celltype_to_newnum = most_common_pairs.set_index('knn_celltype')['cell_type_final'].to_dict()
    celltype_to_newnum = {key + 1: value for key, value in celltype_to_newnum.items()}

    geojson_data = grid_to_geojson_with_scaling(filled_grid, x_bins, y_bins, celltype_to_newnum, scale_factor=10)

    # Save the GeoJSON to a file
    with open(f'{dir_notebook}/coordinates/Region_prediction/Xenium-data-coordinates-filtered_{sample_to_map}.geojson', 'w') as f:
        json.dump(geojson_data, f, indent=2)


print(f'{datetime.now()}: automap done')




## Match cells with automatically generated regions

### Data pre-processing

In [ ]:
adata = sc.read_h5ad(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_MMC_Banksy_annotated_combined.h5ad.gz")

In [ ]:
samples_ids = adata.obs['sample'].unique()
samples_ids

In [ ]:
count = 0
combine_dict = {}

for sample_to_map in samples_ids:
    count += 1
    print(f'{datetime.now()}: mapping of {sample_to_map} ({count}/{len(samples_ids)})')
    geojson  = gpd.read_file(f'{dir_notebook}/coordinates/Region_prediction/Xenium-data-coordinates-filtered_{sample_to_map}.geojson')
    geojson['geometry'][1]
    adata_sub = adata[adata.obs['sample'] == sample_to_map]

    ### Cell mapping
    centroid_gdp = gpd.GeoDataFrame(adata_sub.obs, geometry=gpd.points_from_xy(adata_sub.obs.x_centroid, adata_sub.obs.y_centroid))
    centroid_gdp.index.name = None
    centroid_gdp.crs = 'EPSG:4326'
    matched_cells = gpd.sjoin(centroid_gdp, geojson, predicate='within', how='left')

    mapping_dict_reg = dict(zip(matched_cells['cell_id'], matched_cells['cell_type_newnum_final_right']))
    adata_sub.obs['region_automap'] = adata_sub.obs['cell_id'].map(mapping_dict_reg)

    all_cell_type = adata_sub.obs['region_automap'].unique()
    list_cell_nb = range(0, len(all_cell_type))
    mapping_dict = dict(zip(all_cell_type,list_cell_nb))
    adata_sub.obs['region_automap_newnum'] = adata_sub.obs['region_automap'].map(mapping_dict)

    mapping_dict_reg = dict(zip(adata_sub.obs['cell_id'], adata_sub.obs['region_automap']))
    combine_dict.update(mapping_dict_reg)
    
print(f'{datetime.now()}: dict done')
adata.obs['region_automap_name'] = adata.obs['cell_id'].map(combine_dict)

if 'leiden_colors' in adata.obs:
    adata.obs = adata.obs.drop(columns=['leiden_colors'])

adata.write(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_MMC_Banksy_annotated_automap.h5ad.gz", compression = 'gzip')

adata.obs.to_csv(f"{dir_notebook}/csv/{name_dir}/{name_dir}_MMC_Banksy_annotated_automap.csv.gz",
         compression={'method': 'gzip'})


In [ ]:
all_cell_type = adata.obs['region_automap_name'].unique()
list_cell_nb = range(0, len(all_cell_type))
mapping_dict = dict(zip(all_cell_type,list_cell_nb))
adata.obs['region_automap_num'] = adata.obs['region_automap_name'].map(mapping_dict)
mapping_dict

# Manual Map

In [ ]:
adata = sc.read_h5ad(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_MMC_Banksy_annotated_automap.h5ad.gz") 
# adata = sc.read_h5ad(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_final.h5ad.gz") 

In [ ]:
sample_ids = adata.obs['sample'].unique()
sample_ids

In [ ]:
### Choose sample to map here:
sample_to_map = sample_ids[0]
sample_to_map
BR_df = pd.read_csv(f"coordinates/XE_selection/{sample_to_map}-RL.csv",comment = '#')

In [ ]:
for sample_to_map in sample_ids:
    BR_df = pd.read_csv(f"coordinates/XE_selection/{sample_to_map}-RL.csv",comment = '#')
    # Group the dataframe by the "Selection" column
    grouped = BR_df.groupby('Selection')

    # List to hold GeoJSON features
    features = []

    for name, group in grouped:
        # Create a list of coordinates for each region
        coordinates = [(x, y) for x, y in zip(group['X'], group['Y'])]
        if coordinates[0] != coordinates[-1]:
            coordinates.append(coordinates[0])
        
        # Create a GeoJSON polygon for the region
        polygon = geojson.Polygon([coordinates])
        feature = geojson.Feature(geometry=polygon, properties={"region": name})
        features.append(feature)

    # Create a GeoJSON FeatureCollection
    feature_collection = geojson.FeatureCollection(features)

    # Save the GeoJSON FeatureCollection to a file
    with open(f'coordinates/{sample_to_map}_regions_manual.geojson', 'w') as f:
        geojson.dump(feature_collection, f)

    print(f"GeoJSON {sample_to_map} saved")

In [ ]:
for sample_to_map in sample_ids:
    regions_df = gpd.read_file(f'coordinates/{sample_to_map}_regions_manual.geojson')
    #Region mapping
    adata_region_sub = adata[adata.obs['sample']== sample_to_map]

    centroid_gdp = gpd.GeoDataFrame(adata_region_sub.obs, geometry=gpd.points_from_xy(adata_region_sub.obs.x_centroid, adata_region_sub.obs.y_centroid))
    centroid_gdp.index.name = None
    centroid_gdp.crs = 'EPSG:4326'
    matched_cells = gpd.sjoin(centroid_gdp, regions_df, predicate='within', how='left')
    mapping_dict_reg = dict(zip(matched_cells['cell_id'], matched_cells['region']))
    adata_region_sub.obs['region_manual'] = adata_region_sub.obs['cell_id'].map(mapping_dict_reg)

    all_cell_type = adata_region_sub.obs['region_manual'].unique()
    list_cell_nb = range(0, len(all_cell_type))
    mapping_dict = dict(zip(all_cell_type,list_cell_nb))
    adata_region_sub.obs['region_manual_newnum'] = adata_region_sub.obs['region_manual'].map(mapping_dict)
    mapping_dict_reg = dict(zip(adata_region_sub.obs['cell_id'], adata_region_sub.obs['region_manual']))

    if 'combine_dict_region' not in locals():
        combine_dict_region = {}
        combine_dict_region.update(mapping_dict_reg)   
    else:
        combine_dict_region.update(mapping_dict_reg)


<font size="6"><span style="color:red">From here, go back to process the other samples </span></font>

## Apply combined dictionnary

In [ ]:
### Only run when you are done with all samples automap annotations
adata.obs['region_manual_name'] = adata.obs['cell_id'].map(combine_dict_region)

In [ ]:
adata.obs['region_manual_name'].value_counts()

# Optional annotations

## Type of cells (NGE)

In [ ]:
from module.subclustering_Xe import cell_class_annotation

adata = cell_class_annotation(adata)
adata.obs['cell_class'].value_counts()

## Circascore

In [ ]:
from module.subclustering_Xe import circascore_annot

df = pd.read_parquet(f"{dir_notebook}/csv/{name_dir}/{name_dir}_normalized_counts.parquet")

adata = circascore_annot(adata,df)


In [ ]:
adata.obs['circascore'].value_counts()

## ZT

In [ ]:
adata.obs['ZT'] = adata.obs['sample'].map(lambda name: name.split('-')[-1])

In [ ]:
adata.obs['ZT'].value_counts()

## Genotype

In [ ]:
#TODO

# Final Output

In [ ]:
if 'leiden_colors' in adata.obs:
    adata.obs = adata.obs.drop(columns=['leiden_colors'])

adata.write(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_MMC_Banksy_annotated_automap.h5ad.gz", compression = 'gzip')

adata.obs.to_csv(f"{dir_notebook}/csv/{name_dir}/{name_dir}_MMC_Banksy_annotated_automap.csv.gz",
         compression={'method': 'gzip'})

In [ ]:
adata = sc.read_h5ad(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_MMC_Banksy_annotated_automap.h5ad.gz")

In [ ]:
adata = sc.read_h5ad(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_final.h5ad.gz")

# End of the notebook

Next step : Data vizualization and analysis

[v9C_DataViz_Analysis](./v9C_DataViz_Analysis.ipynb)